In [5]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [6]:
# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)

In [7]:
# URL of the page to scrape
base_url = "https://www.myntra.com/leather-jackets-for-men?rawQuery=leather%20jackets%20for%20men"

In [ ]:
# Function to extract product data
def extract_product_data(url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    products = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-base")))

    product_data = []

    for product in products:
        try:
            brand = product.find_element(By.CLASS_NAME, "product-brand").text
            name = product.find_element(By.CLASS_NAME, "product-product").text
            price = product.find_element(By.CLASS_NAME, "product-discountedPrice").text
            discount = product.find_element(By.CLASS_NAME, "product-discountPercentage").text
            
            product_data.append({'Brand': brand, 'Name': name, 'Price': price, 'Discount': discount})
        except Exception as e:
            print(f"An error occurred while extracting product data: {e}")

    return product_data

# Function to scrape all pages
def scrape_all_pages(base_url):
    all_product_info = []
    driver.get(base_url)
    time.sleep(5)  # Wait for the page to load

    while True:
        page_data = extract_product_data(driver.current_url)
        all_product_info.extend(page_data)

        try:
            # Updated XPath for the "Next" button
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[@class='pagination-next']")))
            next_button.click()
            time.sleep(5)  # Wait for the next page to load
        except Exception as e:
            print("No more pages to scrape or an error occurred:", e)
            break

    return all_product_info

# Function to save data to CSV using pandas
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8')

# Call the functions
all_product_info = scrape_all_pages(base_url)
print(all_product_info)

# Save to CSV only if there is data
if all_product_info:
    save_to_csv(all_product_info, 'scraped_data.csv')
else:
    print("No data to save to CSV.")

# Close the WebDriver
driver.quit()